In [47]:
from sqlalchemy import create_engine
engine = create_engine(f"sqlite:///birdnest.db")
import pandas as pd

from spotclient import Client
import models

db = models.Database()
session = models.get_session()


In [60]:
session = models.get_session()
session.query(models.Playlist).count()



47

In [3]:
def playlist_as_df(cols=None):
    if cols is None:
        af_clause = 'af.date   elif isinstance(cols,str):
        af_clause = cols
    else:
        clause_cols = []
        for c in cols:
            if not c.startswith('af.'):
                c = f"af.{c}"
            clause_cols.append(c)
        af_clause = ','.join(clause_cols)
    
    SQL = f"""
    SELECT
        p.date, 
        p.name playlist_name, p.playlist_id,
        t.name,
        GROUP_CONCAT(a.name,';') artist,
        t.duration_ms,
        {af_clause}
    FROM
        playlist p,
        playlist_track pt,
        track t,
        track_artist ta,
        artist a,
        audio_features af
    where t.track_id = ta.track_id 
         and ta.artist_id = a.artist_id
         and p.playlist_id = pt.playlist_id
         and pt.track_id = t.track_id
         and af.track_id = t.track_id
    group by t.name 
"""

    engine = create_engine(f"sqlite:///birdnest.db")
    return pd.read_sql(SQL, engine)


In [4]:
df = playlist_as_df()
df.columns

Index(['date', 'playlist_name', 'playlist_id', 'name', 'artist', 'duration_ms',
       'features_id', 'track_id', 'analysis_url', 'key', 'mode', 'tempo',
       'time_signature', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence'],
      dtype='object')

In [15]:
def eval_key(key, mode):
    
    PITCH_CLASSES = [
        'C',
        'C♯',
        'D',
        'E♭',
        'E',
        'F',
        'F♯',
        'G',
        'A♭',
        'A',
        'B♭',
        'B'
    ]
    return f"{PITCH_CLASSES[key]} {'major' if mode == '1' else 'minor'}"

df['key_str'] = df.apply(lambda row: eval_key(row['key'],row['mode']),axis=1)


In [30]:
key_counts = df.groupby('key_str')[['playlist_name']].count().reset_index().rename(columns={'playlist_name': 'count', 'key_str': 'key'})

In [46]:
import altair as alt
alt.Chart(alt.Data({'key': ['a','b','c','d','e','f'],'count': [5,4,1,2,6,2]})).mark_bar().encode(x='key',y='count',tooltip=['key', 'count']).properties(width=500)

SchemaValidationError: Invalid specification

        altair.vegalite.v4.schema.core.Data->0, validating 'additionalProperties'

        Additional properties are not allowed ('count', 'key' were unexpected)
        

In [44]:
df[df['key_str'] == 'G minor'][['date','artist', 'name']].sort_values('date',ascending=False)

,date,artist,name
328,2021-03-18,Emily King,Distance
1587,2021-03-18,Sibusile Xaba;Naftali;Fakazile Nkosi;AshK,Umdali
1378,2021-03-18,Monster Rally,Submersible
1563,2021-03-18,Jimmy Carter and Dallas County Green,Travelin'
1136,2021-03-18,Grace Jones,Pull Up To The Bumper - Edit
...,...,...,...
166,2020-04-07,Da' Sha Ra',Bootin' Up
822,2020-04-07,Khun Narin,Lam Phu Thai # 1
819,2020-04-07,Lucinda Williams,Lafayette
1326,2020-04-07,Luke Vibert;BJ Cole,Songs of the Nighlife


In [39]:
def playlist_stats(df):
    """
        Given a dataframe, produce a summarization in the same shape as AudioFeature,
        but aggregated. For now, aggregated as the sum of (each feature * track length)
        but that could become switchable.

        'key', 'mode': probably the mode for each set, and another for the two as a unit?
        'tempo': average
        'time_signature', maybe mode or mode
       'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence'
    """
    for 
    df = df
df['gross_valence'] = df['valence'] * df['duration_ms']
df.groupby('playlist_name').sum()[['gross_valence']].sort_values('gross_valence',ascending=False)


,gross_valence
playlist_name,
JQBX :: Conference of the Birds 2020-05-28,1.037720e+07
JQBX :: Conference of the Birds 2020-12-30,7.494227e+06
JQBX :: Conference of the Birds 2020-11-26,7.371483e+06
JQBX :: Conference of the Birds 2021-03-04,7.078833e+06
JQBX :: Conference of the Birds 2020-05-14,7.031170e+06
JQBX :: Conference of the Birds 2021-01-07,6.920893e+06
JQBX :: Conference of the Birds 2020-09-24,6.818546e+06
💫 JQBX :: Conference of the Birds 2020-04-16,6.804963e+06
JQBX :: Conference of the Birds 2020-12-03,6.630199e+06


In [12]:
print(duration)

0:01:00
